## Prepare calculator

In [ ]:
%load_ext autoreload 
%autoreload 2
import socket
print(socket.gethostname())

# %load_ext autoreload
# %autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )


from prepare_model import prepare_fande_ase_calc

# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 2.0,
'gaussian_sigma_constant': 0.3,
'max_radial': 2,
'max_angular': 2,
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}

N_samples = 10_0
num_grad_steps = 2

fande_calc = prepare_fande_ase_calc(hparams, soap_params, N_samples, num_grad_steps)

In [ ]:
fande_calc.save_predictor("/data1/simulations/ML_models/predictor_garbage.pt")

In [ ]:
import torch
predictor_loaded = torch.load("/data1/simulations/ML_models/predictor_garbage.pt")

In [ ]:
len(fande_calc.predictor.fdm.train_indices[2])

for i in range(14):
    print(len(fande_calc.predictor.fdm.train_indices[i]))

In [ ]:
atoms = fande_calc.predictor.fdm.traj_train[1].copy()
# snap.calc = fande_calc

fande_calc.predictor = predictor_loaded

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
# atoms = traj_test[10].copy()
atoms.set_pbc(True)
atoms.calc = fande_calc

atoms.get_forces()

In [ ]:
os.chdir("/data1/simulations/ML_models/")

In [ ]:
# %%capture c
### MD with fande calc
print('Testing performance with (meta-)dynamics run...')

from fande.ase import FandeCalc
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal

from ase import io

import time

# from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths
from ase.optimize import BFGS
from ase import units
from ase.io import read
import logging
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen


logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

# data_folder = "/home/qklmn/data/"
# traj_test = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "300:400")

# traj_md = read('../results/test/machine_learning/dftb_opt_1000_six_rings.traj', index=":")
# traj_opt = read('../results/test/machine_learning/opt.traj', index=":")

# traj_2000 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/2023-10-19_15%3A10%3A59.497414/md_trajectory.traj",index = "0:10")
atoms = fande_calc.predictor.fdm.traj_train[0].copy()
# snap.calc = fande_calc

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
# atoms = traj_test[10].copy()
atoms.set_pbc(True)
atoms.calc = fande_calc

# moving_atoms = sum(atomic_groups, []) 
# fixed_atoms = list( set(range(264)) - set(moving_atoms) )
# fix_atoms = FixAtoms(indices=fixed_atoms)
# atoms.set_constraint(fix_atoms)


# atoms.calc = FandeCalc(predictor)



# atoms.calc.set_atomic_groups([rings_carbons, rings_hydrogens], titles=["Rings carbons", "Rings hydrogens"])
# atoms.calc.set_forces_errors_plot_file("../results/test/md_runs/forces_errors.png", loginterval=1)
# atoms.calc = LennardJones()

os.makedirs("md_run/", exist_ok=True)

# Verlet dynamics:
# MaxwellBoltzmannDistribution(atoms, temperature_K=300)
# dyn = VelocityVerlet(
#     atoms,
#     dt = 0.1*units.fs,
#     trajectory="md_run/md_test.traj",
#     logfile="md_run/md_log.log",
# )

# dyn = NPT(
#     atoms,
#     # dt = 0.5*units.fs,
#     timestep=0.1,
#     temperature_K=300,
#     externalstress=0.0,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",
# )

# dyn = NPTBerendsen(atoms, timestep=0.1 * units.fs, temperature_K=300,
#                    taut=100 * units.fs, pressure_au=1.01325 * units.bar,
#                    taup=1000 * units.fs, compressibility=4.57e-5 / units.bar,
#                    trajectory="../results/test/md_runs/md_test.traj",
#                    logfile="../results/test/md_runs/md_log.log",)

# import os

# MaxwellBoltzmannDistribution(atoms, temperature_K=300, force_temp=True)
# dyn = NVTBerendsen(atoms, 0.2 * units.fs, 300, taut=0.5*1000*units.fs, 
#                    trajectory="md_run/md_test.traj",   
#                    logfile="md_run/md_log.log")

# dyn.run(100)

# Langevin dynamics:
# https://databases.fysik.dtu.dk/ase/tutorials/md/md.html
# MaxwellBoltzmannDistribution(atoms, temperature_K=100, force_temp=True)
dyn = Langevin(atoms, 0.5*fs, 
               temperature_K=300, #0.1/units.kB, 
               friction=0.1,
        #        fixcm=True, 
               trajectory='md_run/md_testtt_.traj',
               logfile="md_run/md_loggg_.log")

dyn.run(20)

# # Structure optimization:
# dyn = BFGS(
#     atoms,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",)
# dyn.run(fmax=0.1)


print(" ALL JOBS WITHIN PYTHON SCRIPT ARE DONE! ")

In [ ]:
import subprocess

subprocess.check_call(['open', os.getcwd()])
